In [1]:
from random import randint, shuffle
import time
import sys
limit = 10 ** 8
sys.setrecursionlimit(limit)

In [2]:
class Node:
    def __init__(self, key, value=None, left=None, right=None):
        self.key = key
        self.__value = []
        if value:
            self.__value.append(value)
        self.L = left
        self.R = right
        
    @property
    def value(self):
        if self.__value:
            return self.__value
    
    
    @value.setter
    def value(self, value):
        if value:
            self.__value.append(value)
        
    def __repr__(self):
        return f'Node {str(self.key)}'

    
class Tree():
    def __init__(self, root=None):
        self.root = root
        self.row = []
        
    def insert(self, key, value):
        if not self.root:
            self.root = Node(key, value)
        else:
            self.insert_to_node(key, value, self.root)
        
    def insert_to_node(self, key, value, node):
            if node.key == key:
                node.value.append(value)
                return
            if node.key > key:
                if node.L:
                    self.insert_to_node(key, value, node.L)
                else:
                    node.L = Node(key, value)
            else:
                if node.R:
                    self.insert_to_node(key, value, node.R)
                else:
                    node.R = Node(key, value)

    def insert_branch(self, node, branch, side):
        if side == 'R':
            while node.R:
                node = node.R
            node.R = branch
        if side == 'L':
            while node.L:
                node = node.L
            node.L = branch

    def search(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, Node):
                return result
            else:
                return result[0]
        else:
            return None
        
    def search_with_parent(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, Node):
                return (result, None)
            else:
                return result
        else:
            return None
                
    def search_to_node(self, key, node):
        if node.key == key:
            return node
        
        if node.key > key:
            if node.L:                
                result = self.search_to_node(key, node.L)
                if result:
                    if isinstance(result, Node):
                        return(result, node)
                    else:
                        return result
        else:
            if node.R:
                result = self.search_to_node(key, node.R)
                if result:
                    if isinstance(result, Node):
                        return(result, node)
                    else:
                        return result
    
    def remove(self, key):
            node, parent = self.search_with_parent(key)
            if (parent and not node.L and not node.R):
                if node == parent.L:
                    parent.L = None
                else:
                    parent.R = None
                return

            if (parent and node.L and not node.R):
                if node == parent.L:
                    parent.L = node.L
                else:
                    parent.R = node.L
                return

            if (parent and not node.L and node.R):
                if node == parent.L:
                    parent.L = node.R
                else:
                    parent.R = node.R
                return

            if parent:
                if node == parent.L:
                    if node.L:
                        parent.L = node.L
                        self.insert_branch(node.L, node.R, side='R')
                if node == parent.R:
                    if node.R:
                        parent.R = node.R
                        self.insert_branch(node.R, node.L, 'L')
            else:
                if (not node.L and not node.R):
                    self.root = None
                    return
                if node.L and not node.R:
                    self.root = node.L
                    return
                if not node.L and node.R:
                    self.root = node.R
                    return

                self.root = node.L
                self.insert_branch(node.L, node.R, side='R')
                
    def ride(self, order='incr'):   
        node = self.root
        result = []
        def ride_incr(node):
            if node == None:
                return
            ride_incr(node.L)
            result.append(node.key)
            ride_incr(node.R)
        def ride_decr(node):
            if node == None:
                return
            ride_decr(node.R)
            result.append(node.key)
            ride_decr(node.L)
        
        if order == 'incr':    
            ride_incr(node)
        if order == 'decr':
            ride_decr(node)
        return result


# Красно-черное дерево

In [3]:
class BRNode(Node):
    def __init__(self, key, value=None, left=None, right=None, parent=None):
        super().__init__(key, value, left=None, right=None, )
        self.parent = parent
        self.color = 'red'
        if not left:
            self.L = BRList(self)
        if not right:
            self.R = BRList(self)
        
    @property
    def hight(self):        
        if self.L:
            left = self.L.hight
        else:
            left = 1
        if self.R:
            right = self.R.hight
        else:
            right = 1
        if self.color == 'black':
            return max(left, right) + 1
        if self.color == 'red':
            return max(left, right)
    
    @property
    def color(self):
        return self.__color
    
    @color.setter
    def color(self, color):
        self.__color = color
        
    def __repr__(self):
        return f'Node {str(self.key)}-{self.color}'
        

class BRList:
    def __init__(self, parent):
        self.parent = parent
        self.key = None
    
    @property
    def hight(self):
        return 1
    
    @property
    def color(self):
        return 'black'
    
    def __repr__(self):
        return f'List black'
    
        
class BRTree(Tree):
    def __init__(self, root=None):
        super().__init__(root)
    
    def get_grandparent(self, node):
        if ((node != None) and (node.parent != None)):
            return node.parent.parent
        else:
            return None
    
    def get_uncle(self, node):
        grandparent = self.get_grandparent(node)
        if not grandparent:
            return None
        if (node.parent == grandparent.L):
            return grandparent.R
        else:
            return grandparent.L 
        
    def set_color(self, node, color):
        if isinstance(node, BRList):
            return
        node.color = color
        
    def check_rule(self, node):
        if not node.parent:
            if node.color == 'red':
                self.set_color(node, 'black')
            return   
        
        parent = node.parent
        grandparent = self.get_grandparent(node)
        uncle = self.get_uncle(node)
        
    # case_1 parent -> red
        if node.parent.color == 'red':
            if uncle.color == 'red':
#                 print('case 1')
                self.set_color(grandparent, 'red')
                self.set_color(node.parent, 'black')
                self.set_color(uncle, 'black')
                self.check_rule(grandparent)
                return  
            
            # case_2 uncle -> black
            if uncle.color == 'black':
                if (node == parent.R) and (grandparent.L == parent):
#                     print('case 2, LR')
                    self.LR(node)
                    # case_3
#                     print('case 3, BRR-1')
                    self.BRR(parent)
                    return
                if (node == parent.L) and (grandparent.L == parent):
#                     print('case 3, BRR-2')
                    self.BRR(node)
                    return
                if (node == parent.L) and (grandparent.R == parent):
#                     print('case 2, LR')
                    self.RR(node)
                    # case_3
#                     print('case 3, BLR-1')
                    self.BLR(parent)
                    return
                if (node == parent.R) and (grandparent.R == parent):
#                     print('case 3, BLR-2')
                    self.BLR(node)
                    return
            
    def LR(self, node):
        parent = node.parent
        grandparent = self.get_grandparent(node)
        if not grandparent:
            self.root = node
        else: 
            if grandparent.L == parent:
                grandparent.L = node
            else:
                grandparent.R = node
        node.parent = grandparent
        tail = node.L
        parent.R = tail
        tail.parent = parent
        node.L = parent
        parent.parent = node
        
    def RR(self, node):
        parent = node.parent
        grandparent = self.get_grandparent(node)
        if not grandparent:
            self.root = node
        else: 
            if grandparent.L == parent:
                grandparent.L = node
            else:
                grandparent.R = node
        node.parent = grandparent
        tail = node.R
        parent.L = tail
        tail.parent = parent
        node.R = parent
        parent.parent = node
        
    def BRR(self, node):
        parent = node.parent
        grandparent = self.get_grandparent(node)
        if not grandparent.parent:
            self.root = parent
            parent.parent = None
        else:
            if grandparent.parent.R == grandparent:
                grandparent.parent.R = parent
            else:
                grandparent.parent.L = parent
        parent.parent = grandparent.parent
        grandparent.parent = parent
        tail = parent.R
        parent.R = grandparent
        grandparent.L = tail
        tail.parent = grandparent
        self.set_color(parent, 'black')
        self.set_color(grandparent, 'red')
        
    def BLR(self, node):
        parent = node.parent
        grandparent = self.get_grandparent(node)
        if not grandparent.parent:
            self.root = parent
            parent.parent = None
        else:
            if grandparent.parent.R == grandparent:
                grandparent.parent.R = parent
            else:
                grandparent.parent.L = parent
        parent.parent = grandparent.parent
        grandparent.parent = parent
        tail = parent.L
        parent.L = grandparent
        grandparent.R = tail
        tail.parent = grandparent
        self.set_color(parent, 'black')
        self.set_color(grandparent, 'red')
       
    def insert(self, key, value):
        if not self.root:
            self.root = BRNode(key, value)
            self.root.color = 'black'
        else:
            self.insert_to_node(key, value, self.root)

    def insert_to_node(self, key, value, node):
            if node.key == key:
                node.value = value
                return
            if node.key > key:
                if isinstance(node.L, BRNode):
                    self.insert_to_node(key, value, node.L)
                else:
                    node.L = BRNode(key, value, parent=node)
                    self.check_rule(node.L)
            else:
                if isinstance(node.R, BRNode):
                    self.insert_to_node(key, value, node.R)
                else:
                    node.R = BRNode(key, value, parent=node)
                    self.check_rule(node.R)

    def insert_branch(self, node, branch, side):
        if side == 'R':
            while isinstance(node, BRNode):
                node = node.R
            node.R = branch
        if side == 'L':
            while isinstance(node, BRNode):
                node = node.L
            node.L = branch

    def ride(self, order='incr'):   
        node = self.root
        result = []
        if not node:
            return result
        def ride_incr(node):
            if isinstance(node, BRList):
                return
            ride_incr(node.L)
            result.append(node.key)
            ride_incr(node.R)
        def ride_decr(node):
            if isinstance(node, BRList):
                return
            ride_decr(node.R)
            result.append(node.key)
            ride_decr(node.L)
        
        if order == 'incr':    
            ride_incr(node)
        if order == 'decr':
            ride_decr(node)
        return result
    
    
    def search(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, BRNode):
                return result
            else:
                return None
        else:
            return None

    def search_to_node(self, key, node):
        if node.key == key:
            return node
        
        if node.key > key:
            if not isinstance(node.L, BRList):                
                result = self.search_to_node(key, node.L)
                if result:
                    if isinstance(result, BRNode):
                        return result
                    else:
                        return None
        else:
            if not isinstance(node.R, BRList):
                result = self.search_to_node(key, node.R)
                if result:
                    if isinstance(result, BRNode):
                        return result
                    else:
                        return None


# Удаление оставил на потом, не успеваю


#     def remove_2(self, key):
#         # если пустое дерево
#         if not self.root:
#             return
        
#         node = self.search(key)
#         # если узел не найден
#         if not node:
#             return
        
#         ## если удаляемый узел красный
#         if node.color == 'red':
#             # правый и левый ребенок листья
#             if isinstance(node.L, BRList) and isinstance(node.R, BRList):
#                 if node.parent.L == node:
#                     node.parent.L = node.L
#                 else:
#                     node.parent.R = node.R
#                 return
            
#             if  not isinstance(node.L, BRList) and isinstance(node.R, BRList):
#                 child = node.L
#                 child.parent = node.parent
#                 if node.parent.L == node:
#                     node.parent.L = child
#                 else:
#                     node.parent.R = child
#                 return
                            
#             if  isinstance(node.L, BRList) and not isinstance(node.R, BRList):
#                 child = node.R
#                 child.parent = node.parent
#                 if node.parent.L == node:
#                     node.parent.L = child
#                 else:
#                     node.parent.R = child
#                 return

#             if  not isinstance(node.L, BRList) and not isinstance(node.R, BRList):
#                 child = node.L
#                 while not isinstance(child, BRList):
#                     child = child.R
#                 child = child.parent
#                 self.replace_node(node, child) 
#                 temp  = node.L
#                 temp.parent = node.parent
#                 if node.parent.L == child:
#                     child.parent.L = temp
#                 else:
#                     child.parent.R = temp
#                 return

#         ## если удаляемый узел черный
#             if node.color == 'black':
#                 self.delete_case1(node)
        
        
#     def sibling(self, node):
#         if node == node.parent.L:
#             return node.parent.R
#         else:
#             return node.parent.L

#     def replace_node(self, node, child):
#         child.parent = node.parent
#         if node == node.parent.L:
#             node.parent.L = child;
#         else:
#             node.parent.R = child;

#     def delete_one_child(self, node):
#         child = None

#         if isinstance(node.R, BRList):
#             child = node.L 
#         else:
#             child = node.R
#         self.replace_node(n, child)
#         if node.color == 'black':
#             if child.color == 'red':
#                 child.color = 'red'
#             else:
#                 self.delete_case1(child)

#     def delete_case1(self, node):
#         if node.parent:
#             self.delete_case2(node)

#     def delete_case2(self, node):
#         s = sibling(node)
#         if s.color == 'red':
#             node.parent.color = 'red'
#             s.color = 'black'
#             if node == node.parent.L:
#                 rotate_left(node.parent)
#             else:
#                 rotate_right(node.parent)

#         self.delete_case3(node)

#     def rotate_left(self, node):
#         pivot = node.R
#         pivot.parent = node.parent
#         if node.parent:
#             if npde.parent.L == node:
#                 node.parent.L = pivot
#             else:
#                 node.parent.R = pivot

#         node.R = pivot.L
#         if pivot.L:
#             pivot.L.parent = node
#         node.parent = pivot
#         pivot.L = node


#     def rotate_right(self, node):
#         pivot = node.L
#         pivot.parent = node.parent
#         if node.parent:
#             if npde.parent.L == node:
#                 node.parent.L = pivot
#             else:
#                 node.parent.R = pivot

#         node.L = pivot.R
#         if pivot.R:
#             pivot.R.parent = node
#         node.parent = pivot
#         pivot.R = node


#     def delete_case3(self, node):
#         s = sibling(node)
#         if ((node.parent.color == 'black') and
#             (s.color == 'black') and
#             (s.L.color == 'black') and
#             (s.R.color == 'black')):
#             s.color == 'red'
#             self.delete_case1(node.parent)
#         else:
#             self.delete_case4(node)

#     def delete_case4(self, node):
#         s = sibling(node)
#         if ((node.parent.color == 'red') and
#             (s.color == 'black') and
#             (s.L.color == 'black') and
#             (s.R.color == 'black')):
#             s.color == 'red'
#             node.parent.color = 'black'
#         else:
#             self.delete_case5(node)

#     def delete_case5(self, node):
#         s = sibling(node)
#         if s.color == 'black':
#             if ((node == node.parent.L) and
#                 (s.R.color == 'black') and
#                 (s.L.color == 'red')):
#                 s.color == 'red'
#                 s.L.color = 'black'
#                 self.rotate_right(s)
#             elif ((node == node.parent.R) and
#                   (s.L.color == 'black') and
#                   (s.R.color == 'red')):
#                 s.color = 'red'
#                 s.R.color = 'black'
#                 self.rotate_left(s)
#         self.delete_case6(node)   


#     def delete_case6(self, node):
#         s = sibling(node)
#         s.color = node.parent.color
#         node.parent.color = 'black'
#         if (node == node.parent.L):
#             s.R.color = 'black'
#             self.rotate_left(node.parent)
#         else:
#             s.L.color = 'black'
#             self.rotate_right(node.parent)


# АБЛ дерево

In [6]:
class ABLNode(Node):
    def __init__(self, key, value=None, left=None, right=None):
        super().__init__(key, value, left=None, right=None)
        
    @property
    def hight(self):        
        if self.L:
            left = self.L.hight
        else:
            left = -1
        if self.R:
            right = self.R.hight
        else:
            right = -1
        return max(left, right) + 1
    
        
class ABLTree(Tree):
    def __init__(self, root=None):
        super().__init__(root)
        
    def insert(self, key, value):
        if not self.root:
            self.root = ABLNode(key, value)
        else:
            self.insert_to_node(key, value, self.root)
            _, parent = self.search_with_parent(key)
            self.rebalance(parent)
        
    def insert_to_node(self, key, value, node):

            if node.key == key:
                node.value.append(value)
                return
            if node.key > key:
                if node.L:
                    self.insert_to_node(key, value, node.L)
                else:
                    node.L = ABLNode(key, value)
            else:
                if node.R:
                    self.insert_to_node(key, value, node.R)
                else:
                    node.R = ABLNode(key, value)

    def search(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, ABLNode):
                return result
            else:
                return result[0]
        else:
            return None
        
    def search_with_parent(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, ABLNode):
                return (result, None)
            else:
                return result
        else:
            return None
                
    def search_to_node(self, key, node):
        if node.key == key:
            return node
        if node.key > key:
            if node.L:                
                result = self.search_to_node(key, node.L)
                if result:
                    if isinstance(result, ABLNode):
                        return(result, node)
                    else:
                        return result
        else:
            if node.R:
                result = self.search_to_node(key, node.R)
                if result:
                    if isinstance(result, ABLNode):
                        return(result, node)
                    else:
                        return result
    
    def remove(self, key):
        node, parent = self.search_with_parent(key)
        if (parent and not node.L and not node.R):
            if node == parent.L:
                parent.L = None
            return
                
        if (parent and node.L and not node.R):
            if node == parent.L:
                parent.L = node.L
            self.rebalance(parent)
            return
         
        if (parent and not node.L and node.R):
            if node == parent.L:
                parent.L = node.R
            self.rebalance(parent)
            return
                
        if parent:
            if node == parent.L:
                if node.L:
                    parent.L = node.L
                    self.insert_branch(node.L, node.R, side='R')
            if node == parent.R:
                if node.R:
                    parent.R = node.R
                    self.insert_branch(node.R, node.L, 'L')
            self.rebalance(parent)
        else:
            if (not node.L and not node.R):
                self.root = None
                return
            if node.L and not node.R:
                self.root = node.L
                self.rebalance(node.L)
                return
            if not node.L and node.R:
                self.root = node.R
                self.rebalance(node.R)
                return
            
            self.root = node.L
            self.insert_branch(node.L, node.R, side='R')
            self.rebalance(self.root)
    
    def height(self, node):
        if node is None:
            return -1
        else:
            return node.hight
    
    def MPP(self, node):
        _, parent = self.search_with_parent(node.key)
        b = node.L
        c = b.R
        node.L = c
        b.R = node
        if parent:   
            if parent.L:
                if parent.L == node:
                    parent.L = b
            if parent.R:
                if parent.R == node:
                    parent.R = b
        else:
            self.root = b
    
    def MLP(self, node):
        _, parent = self.search_with_parent(node.key)
        b = node.R
        c = b.L
        node.R = c
        b.L = node
        if parent:   
            if parent.L:
                if parent.L == node:
                    parent.L = b
            
            if parent.R:
                if parent.R == node:
                    parent.R = b
        else:
            self.root = b
    
    def BPP(self, node):
        self.MLP(node.L)
        _, parent = self.search_with_parent(node.key)
        self.MPP(node)     

    def BLP(self, node):
        self.MPP(node.R)
        _, parent = self.search_with_parent(node.key)
        self.MLP(node)

    def find_min(self):
        node = self.root
        if node.L:
            return find_min(node.L)

    def rebalance(self, node):
        while node:
            diff = self.height(node.L) - self.height(node.R)
            left = -1
            right = -1
            if node.L:
                if node.L.L:
                    left = node.L.L.hight
                else:
                    left = 0
                if node.L.R:
                    right = node.L.R.hight
                else:
                    left = 0
            if (diff > 1) and (left > right):
                self.MPP(node)
            if (diff > 1) and (left < right):
                self.BPP(node)
            
            if node.R:
                if node.R.L:
                    left = node.R.L.hight
                else:
                    left = 0
                if node.R.R:
                    right = node.R.R.hight
                else:
                    left = 0
            if (diff < -1) and(left < right):
                self.MLP(node)
            if (diff < -1) and(left > right):
                self.BLP(node)
            _, node = self.search_with_parent(node.key)


In [7]:
N = 15000
ordered_arr = [x for x in range(N)]
shuffle_arr = [x for x in range(N)]
shuffle(shuffle_arr)

In [8]:
# добавление элементов в ABL дерево в возрастающем  порядке
start = time.time()
oreder_tree = ABLTree()
for item in ordered_arr:
    oreder_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

118.507938


In [9]:
# добавление элементов в ABL дерево в случайном порядке
start = time.time()
shuffle_tree = ABLTree()
for item in shuffle_arr:
    shuffle_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

137.314205


In [10]:
# поиск элементов в "перемешанном" ABL дереве
start = time.time()
for item in shuffle_arr:
    shuffle_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.065946


In [11]:
# поиск элементов в "возрастающем" ABL дереве
start = time.time()
for item in ordered_arr:
    oreder_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.060703


### Красно-черные деревья 

In [12]:
# добавление элементов в красно-черное дерево в возрастающем  порядке
start = time.time()
oreder_BRTree = BRTree()
for item in ordered_arr:
    oreder_BRTree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

0.223689


In [16]:
# добавление элементов в красно-черное дерево в случайном  порядке
start = time.time()
shuffle_BRTree = BRTree()
for item in shuffle_arr:
    shuffle_BRTree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

0.140450


In [20]:
print('Высота красно-черного дерева -', shuffle_BRTree.root.hight)
print('Высота левой ветви красно-черного дерева -', shuffle_BRTree.root.L.hight)
print('Высота правой ветви красно-черного дерева -', shuffle_BRTree.root.R.hight)

Высота красно-черного дерева - 10
Высота левой ветви красно-черного дерева - 9
Высота правой ветви красно-черного дерева - 9


In [14]:
# поиск элементов в "возрастающем" ABL дереве
start = time.time()
for item in ordered_arr:
    oreder_BRTree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.062674


In [15]:
# поиск элементов в "перемешанном" ABL дереве
start = time.time()
for item in shuffle_arr:
    shuffle_BRTree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.068222
